In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [3]:
print(torch.__version__)

0.4.0


In [4]:
class MusicGenreTrainSet(Dataset):

    def __init__(self, path):
        Dataset.__init__(self)
        self.path = path
        self.dataset = pd.read_csv(path, delimiter=",")
        self.dataset = self.dataset.sample(frac=1)
        self.dataset.replace(to_replace='blues', value=0, inplace=True)
        self.dataset.replace(to_replace='classical', value=1, inplace=True)
        self.dataset.replace(to_replace='country', value=2, inplace=True)
        self.dataset.replace(to_replace='disco', value=3, inplace=True)
        self.dataset.replace(to_replace='hiphop', value=4, inplace=True)
        self.dataset.replace(to_replace='jazz', value=5, inplace=True)
        self.dataset.replace(to_replace='metal', value=6, inplace=True)
        self.dataset.replace(to_replace='pop', value=7, inplace=True)
        self.dataset.replace(to_replace='reggae', value=8, inplace=True)
        self.dataset.replace(to_replace='rock', value=9, inplace=True)
        self.data = self.dataset.iloc[:-200, :-1].as_matrix()
        self.label = self.dataset.iloc[:-200, -1:].as_matrix()

    def __getitem__(self, item):
        return self.data[item, :], self.label[item]

    def __len__(self):
        return len(self.data[:, :])

In [5]:
class MusicGenreTestSet(Dataset):

    def __init__(self, path):
        Dataset.__init__(self)
        self.path = path
        self.dataset = pd.read_csv(path, delimiter=",")
        self.dataset = self.dataset.sample(frac=1)
        self.dataset.replace(to_replace='blues', value=0, inplace=True)
        self.dataset.replace(to_replace='classical', value=1, inplace=True)
        self.dataset.replace(to_replace='country', value=2, inplace=True)
        self.dataset.replace(to_replace='disco', value=3, inplace=True)
        self.dataset.replace(to_replace='hiphop', value=4, inplace=True)
        self.dataset.replace(to_replace='jazz', value=5, inplace=True)
        self.dataset.replace(to_replace='metal', value=6, inplace=True)
        self.dataset.replace(to_replace='pop', value=7, inplace=True)
        self.dataset.replace(to_replace='reggae', value=8, inplace=True)
        self.dataset.replace(to_replace='rock', value=9, inplace=True)
        self.data = self.dataset.iloc[-200:, :-1].as_matrix()
        self.label = self.dataset.iloc[-200:, -1:].as_matrix()

    def __getitem__(self, item):
        return self.data[item, :], self.label[item]

    def __len__(self):
        return len(self.data[:, :])

In [6]:
class Model(torch.nn.Module):

    def __init__(self, input, hidden, output):
        torch.nn.Module.__init__(self)
        self.layer1 = torch.nn.Linear(input, hidden)
        self.layer2 = torch.nn.Linear(hidden, output)
        self.relu = torch.nn.ReLU()

    def forward(self, input):
        out1 = self.relu(self.layer1(input))
        out2 = self.layer2(out1)
        return out2


In [7]:
# GPU
device = torch.device("cuda")

In [8]:
# seed values
torch.manual_seed(100)
np.random.seed(100)

In [17]:
# Train and Test loader
train_loader = MusicGenreTrainSet(path='C:\\Users\\Swapnil.Walke\\PycharmProjects\\PytorchExamples\\clean_data_norm.csv')
test_loader = MusicGenreTestSet(path='C:\\Users\\Swapnil.Walke\\PycharmProjects\\PytorchExamples\\clean_data_norm.csv')
train_loader = DataLoader(train_loader, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_loader, batch_size=32, shuffle=True, num_workers=0)

In [18]:
# Input variables
input = 28
hidden = 50
output = 10

In [19]:
# criterion and entropy
model = Model(input, hidden, output).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [22]:
for epoch in range(504):
    loss = 0.0
    cnt = 1
    for index, data in enumerate(train_loader):
        x, y = data
        x.requires_grad = True
        x = x.float().to(device)
        y = y.squeeze(1).to(device)
        y_pred = model(x)
        loss += criterion(y_pred, y)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        cnt = index
    train_loss = loss.data[0]/cnt
    loss = 0.0
    cnt = 1
    for index, data in enumerate(test_loader):
        x, y = data
        x = x.float().to(device)
        y = y.squeeze(1).to(device)
        y_pred = model(x)
        loss += criterion(y_pred, y)
        cnt = index
    test_loss = loss.data[0]/cnt
    if epoch % 50 == 0 or epoch == 503:
        print(f'Epoch {epoch} : Train Loss : [{train_loss}] Test Loss : [{test_loss}]')

c:\users\swapnil.walke\appdata\local\continuum\anaconda3\envs\pytorch0.4\lib\site-packages\ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app
c:\users\swapnil.walke\appdata\local\continuum\anaconda3\envs\pytorch0.4\lib\site-packages\ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0 : Train Loss : [0.5483368635177612] Test Loss : [1.286982774734497]
Epoch 50 : Train Loss : [0.5575639605522156] Test Loss : [1.1937347650527954]
Epoch 100 : Train Loss : [0.6576098799705505] Test Loss : [1.8422802686691284]
Epoch 150 : Train Loss : [0.5884829759597778] Test Loss : [1.2908546924591064]
Epoch 200 : Train Loss : [0.47698038816452026] Test Loss : [1.364964485168457]
Epoch 250 : Train Loss : [0.5428142547607422] Test Loss : [1.3180692195892334]
Epoch 300 : Train Loss : [0.42463308572769165] Test Loss : [1.6371128559112549]
Epoch 350 : Train Loss : [0.36090266704559326] Test Loss : [1.2485010623931885]
Epoch 400 : Train Loss : [0.3259235620498657] Test Loss : [1.0251888036727905]
Epoch 450 : Train Loss : [0.2715655565261841] Test Loss : [1.395495891571045]
Epoch 500 : Train Loss : [0.32484158873558044] Test Loss : [0.9633712768554688]
Epoch 503 : Train Loss : [0.34206628799438477] Test Loss : [0.9576999545097351]


In [23]:
cnt = 0.0
tot = 0
for index, data in enumerate(test_loader):
    x, y = data
    x = x.float().to(device)
    y = y.squeeze(1).to(device)
    y_pred = model(x)
    preds = y_pred.cpu().detach().numpy().tolist()
    expecteds = y.cpu().detach().numpy().tolist()
    for index, pred in enumerate(preds):
        pred_c = pred.index(max(pred))
        if pred_c == expecteds[index]:
            cnt += 1
        tot += 1
print(f'Accuracy : {cnt/tot}')

Accuracy : 0.83
